# Roosteren van wachtdiensten


We gaan dit keer een rooster voor de wachtdiensten maken. De beschikbaarheden per persoon zijn aangegeven in `wachtdienst_concept.csv`. Een `x` betekent een afwezigheid van de desbetreffende persoon.

We willen een zo goed mogelijk wachtdienst rooster maken, waar we rekening houden met
- Gelijk aantal wachtdiensten over de hele periode
- Gelijke verdeling van feestdagen/weken
- Per week loopt er een iemand wachtdienst
- Een redelijke spreiding tussen het aantal wachtdiensten per persoon (minimaal 3 weken rust na een wachtdienst.)

De feestdagen staan aangegeven in de `feestdagen_concept.csv`

In [1]:
import pandas as pd

feestdagen_df = pd.read_csv("feestdagen_concept.csv", sep=';', index_col=0)
feestdagen_df = feestdagen_df.replace('x', 1).fillna(0).astype(int)

print("Feestdagen")
display(feestdagen_df)

Feestdagen


,5-jan,12-jan,19-jan,26-jan,2-feb,9-feb,16-feb,23-feb,2-mrt,9-mrt,...,20-apr,27-apr,4-mei,11-mei,18-mei,25-mei,1-jun,8-jun,15-jun,22-jun
Week,,,,,,,,,,,,,,,,,,,,,
Aantal feestdagen,0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,1,1,0,0,0,0


In [2]:
wachtdienst_df = pd.read_csv("wachtdienst_concept.csv", sep=';', index_col=0)
wachtdienst_df = wachtdienst_df.replace('x', 0).fillna(1).astype(int)

print("\nWachtdienst beschikbaarheid (1 = beschikbaar, 0 = niet beschikbaar)")
display(wachtdienst_df)


Wachtdienst beschikbaarheid (1 = beschikbaar, 0 = niet beschikbaar)


,5-jan,12-jan,19-jan,26-jan,2-feb,9-feb,16-feb,23-feb,2-mrt,9-mrt,...,20-apr,27-apr,4-mei,11-mei,18-mei,25-mei,1-jun,8-jun,15-jun,22-jun
Naam/Week,,,,,,,,,,,,,,,,,,,,,
M,0,1,1,0,1,1,1,0,1,1,...,1,1,1,0,1,0,1,0,1,1
Sa,1,0,1,0,0,0,0,1,1,1,...,1,0,1,0,1,1,1,1,1,0
Re,1,1,1,1,1,1,1,1,0,1,...,0,1,1,1,0,1,1,1,1,1
St,1,1,1,1,1,1,0,0,1,1,...,1,0,0,1,1,0,1,0,1,1
Ro,0,1,0,1,1,1,1,0,1,1,...,1,1,1,1,1,1,0,1,0,1


In [3]:
print("\nAantal beschikbare personen per week:")
wachtdienst_df.sum(axis=0)


Aantal beschikbare personen per week:


5-jan     3
12-jan    4
19-jan    4
26-jan    3
2-feb     4
9-feb     4
16-feb    3
23-feb    2
2-mrt     4
9-mrt     5
16-mrt    3
23-mrt    4
30-mrt    5
6-apr     4
13-apr    5
20-apr    4
27-apr    3
4-mei     4
11-mei    3
18-mei    4
25-mei    3
1-jun     4
8-jun     3
15-jun    4
22-jun    4
dtype: int64

In [4]:
print("\nAantal beschikbare weken per persoon:")
wachtdienst_df.sum(axis=1)


Aantal beschikbare weken per persoon:


Naam/Week
M     19
Sa    16
Re    21
St    19
Ro    18
dtype: int64

### Sets and indices

- Let $ I $ be the set of persons, indexed by $ i $.
- Let $ W $ be the set of weeks, indexed by $ w $.

### Decision variables

$$
x_{i,w} = 
\begin{cases}
1 & \text{if person } i \text{ is scheduled in week } w, \\
0 & \text{otherwise}.
\end{cases}
$$

### Objective (example: no objective / dummy objective)

If you only need a feasibility model (just find a valid schedule), you can use a dummy objective, e.g.:

$$
\min \; 0
$$

### Constraints

1. **Per week there must only be one person scheduled**

   $$
   \sum_{i \in I} x_{i,w} \le 1 \qquad \forall w \in W.
   $$

2. **Each week must be scheduled**

   $$
   \sum_{i \in I} x_{i,w} \ge 1 \qquad \forall w \in W.
   $$

   Combining 1 and 2, we get exactly one person per week as the final constraint:

   $$
   \sum_{i \in I} x_{i,w} = 1 \qquad \forall w \in W.
   $$

3. **Integrality / binary constraints**

   $$
   x_{i,w} \in \{0,1\} \qquad \forall i \in I, \; \forall w \in W.
   $$

## Eerste versie van het model

In [5]:
import pyomo.environ as pyo

persons = list(wachtdienst_df.index)
weeks = list(wachtdienst_df.columns)

model = pyo.ConcreteModel("Wachtdienst Scheduling")
model.persons = pyo.Set(initialize=persons)
model.weeks = pyo.Set(initialize=weeks)

model.availability = pyo.Param(
    model.persons,
    model.weeks,
    initialize=lambda m, i, w: int(wachtdienst_df.loc[i, w]),
    within=pyo.Binary,
    mutable=True,
)

model.x = pyo.Var(model.persons, model.weeks, within=pyo.Binary)

model.objective = pyo.Objective(expr=0, sense=pyo.minimize)

model.assign_exactly_one = pyo.Constraint(
    model.weeks,
    rule=lambda m, w: sum(m.x[i, w] for i in m.persons) == 1,
)


solver = pyo.SolverFactory("cbc", executable="/usr/bin/cbc")
solver.options["randomSeed"] = 42
result = solver.solve(model, tee=False)
print(f"Solver status: {result.solver.status}, {result.solver.termination_condition} in {result.solver.time}s")

Solver status: ok, optimal in 0.007733821868896484s


Wat gebeurt er eigenlijk??

In [6]:
def print_schedule(model):
    schedule_rows = [
        {"Week": w, "Person": i}
        for w in model.weeks
        for i in model.persons
        if pyo.value(model.x[i, w]) > 0.5
    ]
    schedule_df = pd.DataFrame(schedule_rows).set_index("Week").reindex(weeks)

    print("\nWachtdienstrooster (exacte toewijzing per week)")
    display(schedule_df)

    counts = schedule_df["Person"].value_counts().reindex(model.persons, fill_value=0)
    counts_df = counts.rename("Aantal wachtdiensten").to_frame()
    print("\nAantal wachtdiensten per persoon")
    display(counts_df)

    holiday_flags = feestdagen_df.any(axis=0).astype(int)
    holiday_counts = (
        schedule_df["Person"]
        .to_frame()
        .join(holiday_flags.rename("is_holiday"), on="Week")
        .assign(is_holiday=lambda df: df["is_holiday"].fillna(0).astype(int))
    )
    holiday_counts_df = (
        holiday_counts.loc[holiday_counts["is_holiday"] == 1, "Person"]
        .value_counts()
        .reindex(model.persons, fill_value=0)
        .rename("Aantal feestdagen")
        .to_frame()
    )
    print("\nAantal feestdagen per persoon")
    display(holiday_counts_df)

    conflicts = [
        {"Person": i, "Week": w}
        for i in model.persons
        for w in model.weeks
        if pyo.value(model.x[i, w]) > 0.5 and pyo.value(model.availability[i, w]) == 0
    ]
    if conflicts:
        conflict_df = pd.DataFrame(conflicts).set_index("Week").reindex(weeks).dropna() 
        print("\nConflicten: ingepland maar onbeschikbaar")
        display(conflict_df)
    else:
        print("\nGeen conflicten tussen beschikbaarheid en planning gevonden.")

In [7]:
print_schedule(model)


Wachtdienstrooster (exacte toewijzing per week)


,Person
Week,
5-jan,Ro
12-jan,Re
19-jan,Ro
26-jan,Ro
2-feb,Ro
9-feb,M
16-feb,M
23-feb,M
2-mrt,Re



Aantal wachtdiensten per persoon


,Aantal wachtdiensten
M,9
Sa,2
Re,2
St,0
Ro,12



Aantal feestdagen per persoon


,Aantal feestdagen
M,3
Sa,2
Re,0
St,0
Ro,2



Conflicten: ingepland maar onbeschikbaar


,Person
Week,
5-jan,Ro
19-jan,Ro
23-feb,M
2-mrt,Re
6-apr,Ro
11-mei,Sa
25-mei,M
1-jun,Ro
8-jun,M


We zien het volgende...

- St is 0x ingeroosterd
- Mensen zijn ingeroosterd terwijl ze niet kunnen
- Re en St hebben 0 feestdagen, M heeft er 3...
- Ro en M lopen allebei 3x wachtdienst achter elkaar.

### Dit moet beter... 😠

Laten we de problemen 1 voor 1 oplossen

### St is 0x ingeroosterd

We gaan een nieuwe Decision variable aanmaken, $Z$, die berekent automatisch het maximale aantal assignments. $Z$ is altijd groter dan het aantal assignments, maar omdat we $Z$ minimaliseren, is $Z$ gelijk aan het maximimale aantal assignments.

$Z$ neemt een positieve integer waarde aan..

\begin{align*}
\min \;& Z \\
\text{s.t. }\;& x_{i,w} \le a_{i,w} && \forall i \in I,\; w \in W \\
& \sum_{i \in I} x_{i,w} = 1 && \forall w \in W \\
& \sum_{w \in W} x_{i,w} \le Z && \forall i \in I \\
& x_{i,w} \in \{0,1\} && \forall i \in I,\; w \in W \\
& Z \in \mathbb{Z}_{\ge 0}
\end{align*}

In [8]:
model = pyo.ConcreteModel("Wachtdienst Scheduling")
model.persons = pyo.Set(initialize=persons)
model.weeks = pyo.Set(initialize=weeks)

model.availability = pyo.Param(
    model.persons,
    model.weeks,
    initialize=lambda m, i, w: int(wachtdienst_df.loc[i, w]),
    within=pyo.Binary,
    mutable=True,
)

model.x = pyo.Var(model.persons, model.weeks, within=pyo.Binary)

model.max_assignments = pyo.Var(within=pyo.NonNegativeIntegers)

model.objective = pyo.Objective(expr=model.max_assignments, sense=pyo.minimize)

model.assign_exactly_one = pyo.Constraint(
    model.weeks,
    rule=lambda m, w: sum(m.x[i, w] for i in m.persons) == 1,
)

model.balance_load = pyo.Constraint(
    model.persons,
    rule=lambda m, i: sum(m.x[i, w] for w in m.weeks) <= m.max_assignments,
)


solver = pyo.SolverFactory("cbc", executable="/usr/bin/cbc")
solver.options["randomSeed"] = 42
result = solver.solve(model, tee=False)
print(f"Solver status: {result.solver.status}, {result.solver.termination_condition} in {result.solver.time}s")

Solver status: ok, optimal in 0.016643047332763672s


In [9]:
print_schedule(model)


Wachtdienstrooster (exacte toewijzing per week)


,Person
Week,
5-jan,Sa
12-jan,Re
19-jan,Ro
26-jan,Re
2-feb,Ro
9-feb,M
16-feb,Ro
23-feb,M
2-mrt,Re



Aantal wachtdiensten per persoon


,Aantal wachtdiensten
M,5
Sa,5
Re,5
St,5
Ro,5



Aantal feestdagen per persoon


,Aantal feestdagen
M,1
Sa,1
Re,2
St,3
Ro,0



Conflicten: ingepland maar onbeschikbaar


,Person
Week,
19-jan,Ro
23-feb,M
2-mrt,Re
16-mrt,Sa
18-mei,Re


## Er zijn nog steeds conflicten met beschikbaarheden en wachtdienstroosters

We gaan een nieuw model maken, waardoor iedereen alleen ingeroosterd kan worden als hij beschikbaar is.
We gebruiken hiervoor de decision variable $a_{i,w}$:

$$
a_{i,w} =
\begin{cases}
1, & \text{als persoon } i \text{ beschikbaar is in week } w, \\
0, & \text{anders.}
\end{cases}
$$

Dit zorgt voor de nieuwe model definitie, waarin $x_{i,q} \leq a_{i,w}$, resulterend in: 

\begin{aligned}
\min & Z \\
\text{s.t. } & x_{i,w} \le a_{i,w} && \forall i \in I,; w \in W \\
& \sum_{i \in I} x_{i,w} = 1 && \forall w \in W \\
& \sum_{w \in W} x_{i,w} \le Z && \forall i \in I \\
& x_{i,w} \in {0,1} && \forall i \in I,; w \in W \\
& Z \in \mathbb{Z}_{\ge 0}
\end{aligned}

In [10]:
model = pyo.ConcreteModel("Wachtdienst Scheduling")
model.persons = pyo.Set(initialize=persons)
model.weeks = pyo.Set(initialize=weeks)

model.availability = pyo.Param(
    model.persons,
    model.weeks,
    initialize=lambda m, i, w: int(wachtdienst_df.loc[i, w]),
    within=pyo.Binary,
    mutable=True,
)

model.x = pyo.Var(model.persons, model.weeks, within=pyo.Binary)

model.max_assignments = pyo.Var(within=pyo.NonNegativeIntegers)

model.objective = pyo.Objective(expr=model.max_assignments, sense=pyo.minimize)

model.assign_exactly_one = pyo.Constraint(
    model.weeks,
    rule=lambda m, w: sum(m.x[i, w] for i in m.persons) == 1,
)

model.balance_load = pyo.Constraint(
    model.persons,
    rule=lambda m, i: sum(m.x[i, w] for w in m.weeks) <= m.max_assignments,
)

model.respect_availability = pyo.Constraint(
    model.persons,
    model.weeks,
    rule=lambda m, i, w: m.x[i, w] <= m.availability[i, w],
)


solver = pyo.SolverFactory("cbc", executable="/usr/bin/cbc")
solver.options["randomSeed"] = 42
result = solver.solve(model, tee=False)
print(f"Solver status: {result.solver.status}, {result.solver.termination_condition} in {result.solver.time}s")

Solver status: ok, optimal in 0.009902238845825195s


In [11]:
print_schedule(model)


Wachtdienstrooster (exacte toewijzing per week)


,Person
Week,
5-jan,Re
12-jan,Ro
19-jan,M
26-jan,St
2-feb,M
9-feb,St
16-feb,Ro
23-feb,Re
2-mrt,Ro



Aantal wachtdiensten per persoon


,Aantal wachtdiensten
M,5
Sa,5
Re,5
St,5
Ro,5



Aantal feestdagen per persoon


,Aantal feestdagen
M,0
Sa,3
Re,1
St,1
Ro,2



Geen conflicten tussen beschikbaarheid en planning gevonden.


Alle conflicten zijn opgelost :)

## We hebben nog wel het volgende probleem:
- M heeft 0 feestdagen, terwijl S er 3 heeft.

We gaan weer dezelfde spreiding voor feestdagen bedenken, als dat we met het aantal wachtdiensten hebben gedaan.

Hiervoor introduceren we variabele $F$, die het aantal maximaal aantal feestdagen van alle personen representeert, en dit willen we minimaliseren.

\begin{aligned}
\min ;& F + Z \\
\text{s.t. } &
x_{i,w} \le a_{i,w} && \forall i \in I,; w \in W \\
&
\sum_{i \in I} x_{i,w} = 1 && \forall w \in W \\
&
\sum_{w \in W} x_{i,w} \le Z && \forall i \in I \\
&
\sum_{w \in H} x_{i,w} \le F && \forall i \in I \\
&
x_{i,w} \in {0,1} && \forall i \in I,; w \in W \\
&
Z \in \mathbb{Z}{\ge 0},; F \in \mathbb{Z}{\ge 0}
\end{aligned}


In [12]:
holiday_weeks = [w for w in weeks if feestdagen_df.loc[:, w].any()]

model = pyo.ConcreteModel("Wachtdienst Scheduling")
model.persons = pyo.Set(initialize=persons)
model.weeks = pyo.Set(initialize=weeks)
model.holiday_weeks = pyo.Set(initialize=holiday_weeks)

model.availability = pyo.Param(
    model.persons,
    model.weeks,
    initialize=lambda m, i, w: int(wachtdienst_df.loc[i, w]),
    within=pyo.Binary,
    mutable=True,
)

model.x = pyo.Var(model.persons, model.weeks, within=pyo.Binary)
model.max_assignments = pyo.Var(within=pyo.NonNegativeIntegers)
model.max_holidays = pyo.Var(within=pyo.NonNegativeIntegers)

model.objective = pyo.Objective(
    expr=model.max_assignments + model.max_holidays, sense=pyo.minimize
)
model.assign_exactly_one = pyo.Constraint(
    model.weeks,
    rule=lambda m, w: sum(m.x[i, w] for i in m.persons) == 1,
)
model.balance_load = pyo.Constraint(
    model.persons,
    rule=lambda m, i: sum(m.x[i, w] for w in m.weeks) <= m.max_assignments,
)
model.limit_holidays = pyo.Constraint(
    model.persons,
    rule=lambda m, i: sum(m.x[i, w] for w in m.holiday_weeks) <= m.max_holidays,
)
model.respect_availability = pyo.Constraint(
    model.persons,
    model.weeks,
    rule=lambda m, i, w: m.x[i, w] <= m.availability[i, w],
)

solver = pyo.SolverFactory("cbc", executable="/usr/bin/cbc")
solver.options["randomSeed"] = 42
result = solver.solve(model, tee=False)
print(
    f"Solver status: {result.solver.status}, "
    f"{result.solver.termination_condition} in {result.solver.time}s"
)

Solver status: ok, optimal in 0.011028766632080078s


In [13]:
print_schedule(model)


Wachtdienstrooster (exacte toewijzing per week)


,Person
Week,
5-jan,Sa
12-jan,Ro
19-jan,Sa
26-jan,Re
2-feb,M
9-feb,Re
16-feb,Ro
23-feb,Re
2-mrt,St



Aantal wachtdiensten per persoon


,Aantal wachtdiensten
M,5
Sa,5
Re,5
St,5
Ro,5



Aantal feestdagen per persoon


,Aantal feestdagen
M,1
Sa,1
Re,1
St,2
Ro,2



Geen conflicten tussen beschikbaarheid en planning gevonden.


## Laatste probleem....
```
20-apr	Ro
27-apr	Ro
4-mei	Ro
```

Vindt hij vast niet fijn....

Laten we een nieuwe regel introduceren...
Er moeten minimaal $b$ weken tussen twee wachtdiensten zitten per persoon.




\begin{aligned}
\min ;& F + Z \\
\text{s.t. } &
x_{i,w} \le a_{i,w} && \forall i \in I,; w \in W \\
&
\sum_{i \in I} x_{i,w} = 1 && \forall w \in W \\
&
\sum_{w \in W} x_{i,w} \le Z && \forall i \in I \\
&
\sum_{w \in H} x_{i,w} \le F && \forall i \in I \\
&
\sum_{k=0}^{b} x_{i,w+k} \le 1 && \forall i \in I,; \forall w \in W \text{ met } w+b \le |W|.\\
&
x_{i,w} \in {0,1} && \forall i \in I,; w \in W \\
&
Z \in \mathbb{Z}{\ge 0},; F \in \mathbb{Z}{\ge 0}\\
&
W_{w}^{b} = {w, (w+1),, \ldots , (w+b-1),}
\end{aligned}

In [14]:
b = 3
week_order = list(weeks)

def rest_period_rule(m, i, w):
    start = week_order.index(w)
    end = start + b + 1  # geplande week + b rustweken
    if end > len(week_order):
        return pyo.Constraint.Skip
    window = week_order[start:end]
    return sum(m.x[i, ww] for ww in window if ww in m.weeks) <= 1

In [15]:

model = pyo.ConcreteModel("Wachtdienst Scheduling")
model.persons = pyo.Set(initialize=persons)
model.weeks = pyo.Set(initialize=weeks)
model.holiday_weeks = pyo.Set(initialize=holiday_weeks)

model.availability = pyo.Param(
    model.persons,
    model.weeks,
    initialize=lambda m, i, w: int(wachtdienst_df.loc[i, w]),
    within=pyo.Binary,
    mutable=True,
)

model.x = pyo.Var(model.persons, model.weeks, within=pyo.Binary)
model.max_assignments = pyo.Var(within=pyo.NonNegativeIntegers)
model.max_holidays = pyo.Var(within=pyo.NonNegativeIntegers)

model.objective = pyo.Objective(
    expr=model.max_assignments + model.max_holidays, sense=pyo.minimize
)
model.assign_exactly_one = pyo.Constraint(
    model.weeks,
    rule=lambda m, w: sum(m.x[i, w] for i in m.persons) == 1,
)
model.balance_load = pyo.Constraint(
    model.persons,
    rule=lambda m, i: sum(m.x[i, w] for w in m.weeks) <= m.max_assignments,
)
model.limit_holidays = pyo.Constraint(
    model.persons,
    rule=lambda m, i: sum(m.x[i, w] for w in m.holiday_weeks) <= m.max_holidays,
)
model.respect_availability = pyo.Constraint(
    model.persons,
    model.weeks,
    rule=lambda m, i, w: m.x[i, w] <= m.availability[i, w],
)

model.rest_period = pyo.Constraint(model.persons, model.weeks, rule=rest_period_rule)

solver = pyo.SolverFactory("cbc", executable="/usr/bin/cbc")
solver.options["randomSeed"] = 42
result = solver.solve(model, tee=False)
print(
    f"Solver status: {result.solver.status}, "
    f"{result.solver.termination_condition} in {result.solver.time}s"
)



Solver status: ok, optimal in 0.015182733535766602s


In [16]:
print_schedule(model)


Wachtdienstrooster (exacte toewijzing per week)


,Person
Week,
5-jan,Re
12-jan,Ro
19-jan,Sa
26-jan,St
2-feb,Re
9-feb,M
16-feb,Ro
23-feb,Sa
2-mrt,St



Aantal wachtdiensten per persoon


,Aantal wachtdiensten
M,5
Sa,5
Re,5
St,5
Ro,5



Aantal feestdagen per persoon


,Aantal feestdagen
M,2
Sa,1
Re,1
St,1
Ro,2



Geen conflicten tussen beschikbaarheid en planning gevonden.
